In [3]:
import torch
import sys
sys.path.append("../")
from Models.ResNet import ResnetConfig, ResnetClassifier
from Data.Data import DiffractionDataset

In [7]:
test_dataset=DiffractionDataset("../Data/BravaisLattice_Data.pt", categorical='Bravais Lattice')

In [9]:
#Semi-supervised Model using 50% of the data for labeled training
state_dict=torch.load("../Models/Bravais_model_SGAN.pth")
config = ResnetConfig(
        input_dim = 1,
        output_dim = 14,
        res_dims=[32, 64, 64, 64],
        res_kernel=[5, 7, 17, 13],
        res_stride=[4, 4, 5, 3],
        num_blocks=[2, 2, 2, 2],
        first_kernel_size = 13,
        first_stride = 1,
        first_pool_kernel_size = 7,
        first_pool_stride = 7,
    )
net=ResnetClassifier(config)
net.eval().float()
net.load_state_dict(state_dict['model_state_dict'])

<All keys matched successfully>

In [10]:
logits=net(test_dataset.data).logits
predictions=torch.flatten(torch.argmax(logits, dim=-1))
print(predictions)
print(test_dataset.labels)

tensor([ 9,  0,  3,  5,  9, 13,  3,  0,  3, 11])
tensor([ 9,  0,  3, 12,  9,  4,  3,  0,  3, 11])


In [11]:
accuracy=(torch.sum((predictions==test_dataset.labels))/len(predictions))
print(accuracy)

tensor(0.8000)


In [12]:
#Supervised Model using 90% of the data
supervised=ResnetClassifier(config)
supervised.eval().float()
state_dict=torch.load("../Models/Bravais_model_supervised.pth")
supervised.load_state_dict(state_dict['model_state_dict'])

<All keys matched successfully>

In [13]:
logits=supervised(test_dataset.data).logits
predictions=torch.flatten(torch.argmax(logits, dim=-1))
print(predictions)
print(test_dataset.labels)

tensor([12,  0,  3, 11,  9, 13,  3,  0,  3, 11])
tensor([ 9,  0,  3, 12,  9,  4,  3,  0,  3, 11])


In [14]:
accuracy=(torch.sum((predictions==test_dataset.labels))/len(predictions))
print(accuracy)

tensor(0.7000)
